In [41]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import featuretools as ft
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
print(os.getcwd())

# Any results you write to the current directory are saved as output.

['Train.csv', 'Test.csv']
/kaggle/working


In [42]:
train = pd.read_csv("../input/Train.csv")
test = pd.read_csv("../input/Test.csv")
train.head

<bound method NDFrame.head of      Item_Identifier        ...          Item_Outlet_Sales
0              FDA15        ...                  3735.1380
1              DRC01        ...                   443.4228
2              FDN15        ...                  2097.2700
3              FDX07        ...                   732.3800
4              NCD19        ...                   994.7052
5              FDP36        ...                   556.6088
6              FDO10        ...                   343.5528
7              FDP10        ...                  4022.7636
8              FDH17        ...                  1076.5986
9              FDU28        ...                  4710.5350
10             FDY07        ...                  1516.0266
11             FDA03        ...                  2187.1530
12             FDX32        ...                  1589.2646
13             FDS46        ...                  2145.2076
14             FDF32        ...                  1977.4260
15             FDP49      

> Storing target variable in variable sales and saving id variables in test_item_id and test_outlest_id

In [43]:
test_item_id = test['Item_Identifier']
test_outlet_id = test['Outlet_Identifier']
sales = train['Item_Outlet_Sales']
train.drop(['Item_Outlet_Sales'], axis=1, inplace=True)

> combine train and test sets and check for missing values

In [44]:
combi = train.append(test, ignore_index=True)
combi.isnull().sum()

Item_Identifier                 0
Item_Weight                  2439
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  4016
Outlet_Location_Type            0
Outlet_Type                     0
dtype: int64

> Missing values treatment

In [45]:
combi['Item_Weight'].fillna(combi['Item_Weight'].mean(), inplace=True)
combi['Outlet_Size'].fillna("missing", inplace=True)

> Data preprocessing

In [46]:
combi['Item_Fat_Content'].value_counts()

Low Fat    8485
Regular    4824
LF          522
reg         195
low fat     178
Name: Item_Fat_Content, dtype: int64

In [47]:
fat_content_dict = {'Low Fat':0, 'Regular':1, 'LF':0, 'reg':1, 'low fat':0}

combi['Item_Fat_Content'] = combi['Item_Fat_Content'].replace(fat_content_dict, regex=True)

> Feature Engineering using Feature Tools

In [48]:
combi.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDA15,9.30,0,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1
1,DRC01,5.92,1,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2
2,FDN15,17.50,0,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1
3,FDX07,19.20,1,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,missing,Tier 3,Grocery Store
4,NCD19,8.93,0,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1


In [49]:
combi['id'] = combi['Item_Identifier'] + combi['Outlet_Identifier']
combi.drop(['Item_Identifier'], axis=1, inplace=True)

In [50]:
# creating and entity set 'es'
es = ft.EntitySet(id = 'sales')

# adding a dataframe 
es.entity_from_dataframe(entity_id = 'bigmart', dataframe = combi, index = 'id')

Entityset: sales
  Entities:
    bigmart [Rows: 14204, Columns: 11]
  Relationships:
    No relationships

In [51]:
es.normalize_entity(base_entity_id='bigmart', new_entity_id='outlet', index = 'Outlet_Identifier', 
additional_variables = ['Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type'])

Entityset: sales
  Entities:
    bigmart [Rows: 14204, Columns: 7]
    outlet [Rows: 10, Columns: 5]
  Relationships:
    bigmart.Outlet_Identifier -> outlet.Outlet_Identifier

In [52]:
feature_matrix, feature_names = ft.dfs(entityset=es, 
target_entity = 'bigmart', 
max_depth = 2, 
verbose = 1, 
n_jobs = 3)

Built 37 features
EntitySet scattered to workers in 0.602 seconds
Elapsed: 00:01 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 11/11 chunks


In [53]:
feature_matrix.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,outlet.Outlet_Establishment_Year,outlet.Outlet_Size,outlet.Outlet_Location_Type,outlet.Outlet_Type,outlet.SUM(bigmart.Item_Weight),outlet.SUM(bigmart.Item_Fat_Content),outlet.SUM(bigmart.Item_Visibility),outlet.SUM(bigmart.Item_MRP),outlet.STD(bigmart.Item_Weight),outlet.STD(bigmart.Item_Fat_Content),outlet.STD(bigmart.Item_Visibility),outlet.STD(bigmart.Item_MRP),outlet.MAX(bigmart.Item_Weight),outlet.MAX(bigmart.Item_Fat_Content),outlet.MAX(bigmart.Item_Visibility),outlet.MAX(bigmart.Item_MRP),outlet.SKEW(bigmart.Item_Weight),outlet.SKEW(bigmart.Item_Fat_Content),outlet.SKEW(bigmart.Item_Visibility),outlet.SKEW(bigmart.Item_MRP),outlet.MIN(bigmart.Item_Weight),outlet.MIN(bigmart.Item_Fat_Content),outlet.MIN(bigmart.Item_Visibility),outlet.MIN(bigmart.Item_MRP),outlet.MEAN(bigmart.Item_Weight),outlet.MEAN(bigmart.Item_Fat_Content),outlet.MEAN(bigmart.Item_Visibility),outlet.MEAN(bigmart.Item_MRP),outlet.COUNT(bigmart),outlet.NUM_UNIQUE(bigmart.Item_Type),outlet.MODE(bigmart.Item_Type)
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
DRA12OUT010,11.600000,0,0.068535,Soft Drinks,143.0154,OUT010,1998,missing,Tier 3,Grocery Store,11768.655000,330,94.293418,130572.7618,4.675070,0.479301,0.073604,62.010835,21.350000,1,0.313935,266.6884,0.112759,0.599012,0.776902,0.104693,4.610000,0,0.0,32.6558,12.722870,0.356757,0.101939,141.159742,925,16,Fruits and Vegetables
DRA12OUT013,11.600000,0,0.040912,Soft Drinks,142.3154,OUT013,1987,High,Tier 3,Supermarket Type1,19859.980000,549,93.555174,219172.4492,4.650214,0.478213,0.044005,62.140848,21.350000,1,0.185913,266.6884,0.104392,0.613449,0.759033,0.130888,4.555000,0,0.0,31.4900,12.788139,0.353509,0.060242,141.128428,1553,16,Fruits and Vegetables
DRA12OUT017,11.600000,0,0.041178,Soft Drinks,140.3154,OUT017,2007,missing,Tier 2,Supermarket Type1,19722.750000,544,94.342210,217561.3500,4.655234,0.477922,0.044152,62.295513,21.350000,1,0.188620,266.8884,0.106563,0.617805,0.774783,0.129950,4.555000,0,0.0,32.0900,12.782080,0.352560,0.061142,140.998931,1543,16,Snack Foods
DRA12OUT018,11.600000,0,0.041113,Soft Drinks,142.0154,OUT018,2009,Medium,Tier 3,Supermarket Type2,19794.425000,547,92.723425,217987.3906,4.650874,0.478308,0.044489,62.022851,21.350000,1,0.188323,266.3226,0.102602,0.612046,0.783017,0.133528,4.555000,0,0.0,31.8900,12.803638,0.353816,0.059976,141.000899,1546,16,Fruits and Vegetables
DRA12OUT027,12.792854,0,0.040748,Soft Drinks,140.0154,OUT027,1985,Medium,Tier 3,Supermarket Type3,19944.059742,551,94.075671,219838.2488,0.000000,0.478189,0.044228,62.059660,12.792854,1,0.186650,265.2884,0.000000,0.613802,0.774028,0.129638,12.792854,0,0.0,31.2900,12.792854,0.353432,0.060344,141.012347,1559,16,Fruits and Vegetables


> here is one issue with this dataframe – it is not sorted properly. We will have to sort it based on the id variable from the combi dataframe.



In [54]:
feature_matrix = feature_matrix.reindex(index=combi['id'])
feature_matrix = feature_matrix.reset_index()
feature_matrix.head()

,id,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,outlet.Outlet_Establishment_Year,outlet.Outlet_Size,outlet.Outlet_Location_Type,outlet.Outlet_Type,outlet.SUM(bigmart.Item_Weight),outlet.SUM(bigmart.Item_Fat_Content),outlet.SUM(bigmart.Item_Visibility),outlet.SUM(bigmart.Item_MRP),outlet.STD(bigmart.Item_Weight),outlet.STD(bigmart.Item_Fat_Content),outlet.STD(bigmart.Item_Visibility),outlet.STD(bigmart.Item_MRP),outlet.MAX(bigmart.Item_Weight),outlet.MAX(bigmart.Item_Fat_Content),outlet.MAX(bigmart.Item_Visibility),outlet.MAX(bigmart.Item_MRP),outlet.SKEW(bigmart.Item_Weight),outlet.SKEW(bigmart.Item_Fat_Content),outlet.SKEW(bigmart.Item_Visibility),outlet.SKEW(bigmart.Item_MRP),outlet.MIN(bigmart.Item_Weight),outlet.MIN(bigmart.Item_Fat_Content),outlet.MIN(bigmart.Item_Visibility),outlet.MIN(bigmart.Item_MRP),outlet.MEAN(bigmart.Item_Weight),outlet.MEAN(bigmart.Item_Fat_Content),outlet.MEAN(bigmart.Item_Visibility),outlet.MEAN(bigmart.Item_MRP),outlet.COUNT(bigmart),outlet.NUM_UNIQUE(bigmart.Item_Type),outlet.MODE(bigmart.Item_Type)
0,FDA15OUT049,9.30,0,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,19844.655,547,91.450099,218802.9588,4.650796,0.478027,0.043924,62.144594,21.35,1,0.187850,266.4884,0.099024,0.616228,0.790782,0.126294,4.555,0,0.0,32.4558,12.803003,0.352903,0.059000,141.163199,1550,16,Fruits and Vegetables
1,DRC01OUT018,5.92,1,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,19794.425,547,92.723425,217987.3906,4.650874,0.478308,0.044489,62.022851,21.35,1,0.188323,266.3226,0.102602,0.612046,0.783017,0.133528,4.555,0,0.0,31.8900,12.803638,0.353816,0.059976,141.000899,1546,16,Fruits and Vegetables
2,FDN15OUT049,17.50,0,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,19844.655,547,91.450099,218802.9588,4.650796,0.478027,0.043924,62.144594,21.35,1,0.187850,266.4884,0.099024,0.616228,0.790782,0.126294,4.555,0,0.0,32.4558,12.803003,0.352903,0.059000,141.163199,1550,16,Fruits and Vegetables
3,FDX07OUT010,19.20,1,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,missing,Tier 3,Grocery Store,11768.655,330,94.293418,130572.7618,4.675070,0.479301,0.073604,62.010835,21.35,1,0.313935,266.6884,0.112759,0.599012,0.776902,0.104693,4.610,0,0.0,32.6558,12.722870,0.356757,0.101939,141.159742,925,16,Fruits and Vegetables
4,NCD19OUT013,8.93,0,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,19859.980,549,93.555174,219172.4492,4.650214,0.478213,0.044005,62.140848,21.35,1,0.185913,266.6884,0.104392,0.613449,0.759033,0.130888,4.555,0,0.0,31.4900,12.788139,0.353509,0.060242,141.128428,1553,16,Fruits and Vegetables


> Model Building

In [55]:
from catboost import CatBoostRegressor


>CatBoost requires all the categorical variables to be in the string format. So, we will convert the categorical variables in our data to string first:

In [56]:
categorical_features = np.where(feature_matrix.dtypes == 'object')[0]

for i in categorical_features:
    feature_matrix.iloc[:,i] = feature_matrix.iloc[:,i].astype('str')

> Split in train and test split

In [57]:
feature_matrix.drop(['id'], axis=1, inplace=True)
train = feature_matrix[:8523]
test = feature_matrix[8523:]
train.drop(['Outlet_Identifier'], axis=1, inplace=True)
test.drop(['Outlet_Identifier'], axis=1, inplace=True)
categorical_features = np.where(train.dtypes == 'object')[0]
print(categorical_features)

[ 3  6  7  8 35]


In [58]:
from sklearn.model_selection import train_test_split

# splitting train data into training and validation set
xtrain, xvalid, ytrain, yvalid = train_test_split(train, sales, test_size=0.25, random_state=11)

In [59]:
model_cat = CatBoostRegressor(iterations=100, learning_rate=0.3, depth=6, eval_metric='RMSE', random_seed=7)

# training model
model_cat.fit(xtrain, ytrain, cat_features=categorical_features, use_best_model=True)
# validation score
model_cat.score(xvalid, yvalid)

You should provide test set for use best model. use_best_model parameter swiched to false value.


0:	learn: 2114.4389034	total: 64.5ms	remaining: 6.38s
1:	learn: 1702.9810381	total: 76.6ms	remaining: 3.75s
2:	learn: 1436.7201311	total: 89.4ms	remaining: 2.89s
3:	learn: 1287.1422746	total: 100ms	remaining: 2.4s
4:	learn: 1194.6282737	total: 114ms	remaining: 2.16s
5:	learn: 1139.4485158	total: 127ms	remaining: 1.99s
6:	learn: 1107.3891181	total: 139ms	remaining: 1.85s
7:	learn: 1092.6732703	total: 150ms	remaining: 1.72s
8:	learn: 1084.6476015	total: 162ms	remaining: 1.64s
9:	learn: 1077.2044663	total: 175ms	remaining: 1.58s
10:	learn: 1073.4339740	total: 189ms	remaining: 1.53s
11:	learn: 1072.0152203	total: 197ms	remaining: 1.44s
12:	learn: 1069.1789740	total: 207ms	remaining: 1.38s
13:	learn: 1068.9751029	total: 215ms	remaining: 1.32s
14:	learn: 1068.0602000	total: 225ms	remaining: 1.27s
15:	learn: 1067.6439283	total: 231ms	remaining: 1.21s
16:	learn: 1067.5700966	total: 245ms	remaining: 1.19s
17:	learn: 1067.0678308	total: 254ms	remaining: 1.16s
18:	learn: 1066.3501879	total: 260ms

1088.4021709875988